<a href="https://colab.research.google.com/github/Biel-PS/AdvancedFluidMechanics/blob/main/0-Intro/Assignment0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 0: Conical tank emptying time

Let's calculate how long it takes to [empty a conical tank](https://www.youtube.com/watch?v=2jQ1jA8uJuU)

The simplification of the continuity equation leads, for a generic $h$, to

$$
u_1 D_h^2 = u_1 \left(d + 2h \tan \theta \right)^2 = u_2 d^2 \tag{1}
$$

$$
\Rightarrow \; u_2 = u_1\left(1 + 2\frac{h}{d} \tan \theta \right)^2 \tag{2}
$$

On the other hand, Bernoulli's equation, considering quasi-steady flow and without viscosity,

$$
u_1^2 + 2gh = u_2^2 = u_1^2 \left(1 + 2\frac{h}{d} \tan \theta \right)^4 \tag{3}
$$

$$
\Rightarrow \; u_1^2 \left[ \left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1 \right] = 2gh \tag{4}
$$

$$
\Rightarrow \; u_1 = \left[ \frac{2gh}{\left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1} \right]^\frac{1}{2} = -\frac{\textrm{d} h}{\textrm{d} t} \tag{5}
$$

$$
\Rightarrow \; \boxed{ dt = - \left[ \frac{\left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1}{2gh}\right]^\frac{1} {2} \textrm{d} h} \tag{6}
$$

This can be written dimensionless with a time characteristic $\tau = \sqrt{\frac{d}{2g}}$, a change to dimensionless variable $\xi = \frac{h}{d}$, and a parameter $A = 2\tan \theta$, becoming

$$
\frac{\textrm{d} t}{\tau} = - \left[ \frac{\left(1 + A \xi \right)^4 -1}{\xi}\right]^\frac{1 }{2} \textrm{d} \xi \tag{7}
$$

Use sympy to analytically compute this time, making an estimation with  $d \ll h$ so that it can be supposed that  $\xi \gg 1$.

# **Assigment 0 solution**
by Group 1: Tobia Pedrotti, Savely Surnachev, Jinuk Lim and Biel Pujadas

Firstly, the libraries that are going to be used are imported. In this case, the symbolic mathematics in python library (*sympy*) will be used for the algebraic operations, and solvers needed to accomplish the solution. The numerical python library (*numpy*) will be used for its methods, more specifically, for its mathematical operators. Finally, the Mpmath library will be used to solve the problem numerically.

In [160]:
import sympy as sp
import mpmath as mp
import numpy as np
sp.init_printing()

Once the libraries are imported, the symbolical variables that will be used are defined.

In [161]:
t, tau, A, xi, xi_i, xi_f, t_i, t_f = sp.symbols('t, tau, A, xi, xi_i, xi_f, t_i, t_f')



Note that the parameters t, τ, ξ and A are presented in equation (7). The parameters "xi_i" ($ξ_i$) and "xi_f" ($ξ_f$) are the initial and final values for the variable $ξ$. Finally, the parameter $t_{f}$ is the time at which $ξ_f$ is reached.

---



Next step is to define the expression that is going to be solved. In this case, the adimensional form of the state equation (7) has been chosen.


In [169]:
expr= - (tau*((1+A*xi)**2)/sp.sqrt(xi))
display(expr)

            2 
-τ⋅(A⋅ξ + 1)  
──────────────
      √ξ      

Now the equation is solved by integration. And the analytical expression of the problem is found.

In [170]:
Result = sp.integrate(expr,(xi,xi_i,xi_f))
Analitical_Expression = sp.Eq(Result,t_f - t_i)
display(Analitical_Expression)

    ⎛   2    5/2          3/2            ⎞     ⎛   2   5/2         3/2           ⎞           
    ⎜2⋅A ⋅ξ_f      4⋅A⋅ξ_f          _____⎟     ⎜2⋅A ⋅ξᵢ      4⋅A⋅ξᵢ          ____⎟           
- τ⋅⎜─────────── + ────────── + 2⋅╲╱ ξ_f ⎟ + τ⋅⎜────────── + ───────── + 2⋅╲╱ ξᵢ ⎟ = t_f - tᵢ
    ⎝     5            3                 ⎠     ⎝    5            3               ⎠           

Now, the problem will be solved numerically.

First numerical values for the different variables are assigned

In [125]:
theta = 20*np.pi/180
AVal = 2*np.tan(theta)
d = 0.01
g = 9.81
tauVal = np.sqrt(d/(2*g))

xi_iVal = 0.2
xi_fVal = 0

Then the values are substituted in the expression


In [126]:
changes = {tau:tauVal,A:AVal}
expNum = sp.lambdify(xi,expr.subs(changes))
display(expNum)

<function _lambdifygenerated(xi)>

Then the equation is solved using Mpmath method *mp. quad* which integrates the function using a type of Gaussian quadrature.

In [127]:
mp.dps = 15 # decimal precision.
NumResult = (mp.quad(expNum , [xi_iVal, xi_fVal]))
print(NumResult)

0.0222382345258811


The results can be compared by replacing the values in the analytically found expression. If the procedure has been correctly done, the results should be equal.


In [172]:
theta = 20*np.pi/180
AVal = 2*np.tan(theta)
d = 0.01
g = 9.81
tauVal = np.sqrt(d/(2*g))

xi_iVal = 0.2
xi_fVal = 0
Changes_To_analitical = {tau:tauVal,A:AVal,xi_i:xi_iVal,xi_f:xi_fVal,t_i:0}
AnalyticalResult = Analitical_Expression.subs(Changes_To_analitical)
display(AnalyticalResult)

0.0222382345312374 = t_f

If the difference between the results is done, it should be expected a value very close to 0. This value can be used to measure the error between results.

In [175]:
Error = AnalyticalResult.lhs - NumResult
display (Error)

5.35625710451626e-12

It can be seen that the value of error is very small, this means that the numerical and analytical solutions are nearly identical.